In [81]:
target_file =  "kube.tf.yaml"
target_folder = "yaml_tree/commons"

In [82]:
import os
import yaml

def replace_vars_with_defaults(
    base_dir: str,
    variables_filename: str = "variables.yaml",
    chosen_filename: str   = "chosen.yaml"
) -> str:
    """
    Given a folder `base_dir` that contains both variables_filename
    and chosen_filename, this will:
      1. Load all defaults from variables_filename
         (either as a top-level list or under a "variable:" key)
      2. Read chosen_filename as text
      3. Replace every occurrence of ${var.<name>} with its default
    Saves the modified text to moded.yaml and returns it.
    """
    # Resolve folder path
    base_dir = os.path.abspath(base_dir)
    print(f"[DEBUG] base_dir resolved to: {base_dir}")

    # Paths to each file
    vars_path   = os.path.join(base_dir, variables_filename)
    chosen_path = os.path.join(base_dir, chosen_filename)
    print(f"[DEBUG] variables file path: {vars_path}")
    print(f"[DEBUG] chosen file path:   {chosen_path}")

    # Load and normalize your variables.yaml
    with open(vars_path, 'r') as vf:
        data = yaml.safe_load(vf) or {}
    print(f"[DEBUG] raw loaded data type: {type(data)}, content: {data}")

    if isinstance(data, dict) and 'variable' in data and isinstance(data['variable'], list):
        vars_list = data['variable']
    elif isinstance(data, list):
        vars_list = data
    else:
        print("[DEBUG] Unexpected variables.yaml structure – treating as empty list")
        vars_list = []

    print(f"[DEBUG] normalized vars_list ({len(vars_list)} items): {vars_list}")

    # Build defaults mapping
    defaults = {}
    for entry in vars_list:
        if isinstance(entry, dict):
            for var_name, props in entry.items():
                if isinstance(props, dict) and 'default' in props:
                    defaults[var_name] = props['default']
    print(f"[DEBUG] defaults mapping: {defaults}")

    # Read the chosen file
    with open(chosen_path, 'r') as cf:
        content = cf.read()
    print(f"[DEBUG] initial chosen content preview:\n{content[:200].replace(chr(10), '\\n')}...")

    # Perform replacements
    for name, default_val in defaults.items():
        placeholder = f"${{var.{name}}}"    # exactly matches ${var.<name>}
        if placeholder in content:
            print(f"[DEBUG] Replacing {placeholder} → {default_val}")
            content = content.replace(placeholder, str(default_val))
        else:
            print(f"[DEBUG] Placeholder {placeholder} not found; skipping")

    # Write out the modified file
    output_path = os.path.join(base_dir, "moded.yaml")
    with open(output_path, 'w') as of:
        of.write(content)
    print(f"[DEBUG] Modified content written to: {output_path}")

    return content


In [83]:
if __name__ == "__main__":
    # Suppose your files live in "../terraform-config"

    result = replace_vars_with_defaults(base_dir=target_folder,
                                        variables_filename="variables.tf.yaml",
                                        chosen_filename=target_file)
    print(result)


[DEBUG] base_dir resolved to: c:\Users\boadeyem\OneDrive - Indiana University\Documents\Masters-Career Documents\Portfolio\Projects\Gen3\yaml_tree\commons
[DEBUG] variables file path: c:\Users\boadeyem\OneDrive - Indiana University\Documents\Masters-Career Documents\Portfolio\Projects\Gen3\yaml_tree\commons\variables.tf.yaml
[DEBUG] chosen file path:   c:\Users\boadeyem\OneDrive - Indiana University\Documents\Masters-Career Documents\Portfolio\Projects\Gen3\yaml_tree\commons\kube.tf.yaml
[DEBUG] raw loaded data type: <class 'dict'>, content: {'variable': [{'vpc_name': {'default': 'Commons1'}}, {'iam_role_name': {'default': 'csoc_adminvm'}}, {'vpc_cidr_block': {'default': '172.24.17.0/20'}}, {'vpc_flow_logs': {'default': False}}, {'vpc_flow_traffic': {'default': 'ALL'}}, {'aws_region': {'default': 'us-east-1'}}, {'aws_cert_name': {'default': 'AWS-CERTIFICATE-NAME'}}, {'csoc_account_id': {'default': '433568766270'}}, {'peering_cidr': {'default': '10.128.0.0/20'}}, {'fence_db_size': {'def